In [26]:
from keras.applications import VGG19

# MobileNet was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 
batch_size = 16

# Re-loads the MobileNet model without the top or FC layers
vgg19 = VGG19(weights = 'imagenet',include_top = False,input_shape = (img_rows, img_cols, 3)) #include_top = False to customize our input for the model


# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in vgg19.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(vgg19.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 Conv2D False
11 MaxPooling2D False
12 Conv2D False
13 Conv2D False
14 Conv2D False
15 Conv2D False
16 MaxPooling2D False
17 Conv2D False
18 Conv2D False
19 Conv2D False
20 Conv2D False
21 MaxPooling2D False


In [27]:
def addTopModelvgg19(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [28]:
import os
print(len(os.listdir('./Dataset/Train')))
print(os.listdir('./Dataset/Train'))

10
['Angrybirds', 'Ben', 'Chicken_little', 'Cinderella', 'Godzilla', 'Mickey_mouse', 'Naruto', 'Pokemon', 'Popeye', 'Tom and Jerry']


In [29]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Set our class number to 3 (Young, Middle, Old)
num_classes = 10

FC_Head = addTopModelvgg19(vgg19, num_classes)

model = Model(inputs = vgg19.input, outputs = FC_Head)

print(model.summary())

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [30]:
from keras.preprocessing.image import ImageDataGenerator
TRAIN_DIR = './Dataset/Train'

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.3)

train_generator= datagen.flow_from_directory(TRAIN_DIR,
                                             subset='training',
                                             target_size=(img_rows, img_cols),
                                             batch_size=batch_size,
                                             class_mode='categorical')
validation_datagen=datagen.flow_from_directory(TRAIN_DIR,
                                               subset='validation')

Found 1485 images belonging to 10 classes.
Found 630 images belonging to 10 classes.


In [31]:
%%time
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
                   
checkpoint = ModelCheckpoint("dscomp.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 15,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.1,
                              patience = 2,
                              verbose = 1,
                              min_delta = 0.000001)
# we put our call backs into a callback list
callbacks = [earlystop, checkpoint, reduce_lr]

# Note we use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 1485  
nb_validation_samples = 630
epochs = 10

history = model.fit(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_datagen,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/10
92/92 [==============================] - ETA: 0s - loss: 2.1937 - accuracy: 0.2144WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 39 batches). You may need to use the repeat() function when building your dataset.

Epoch 00001: val_loss improved from inf to 2.03909, saving model to dscomp.h5
92/92 [==============================] - 34s 364ms/step - loss: 2.1937 - accuracy: 0.2144 - val_loss: 2.0391 - val_accuracy: 0.2683
Epoch 2/10
92/92 [==============================] - 20s 218ms/step - loss: 1.8670 - accuracy: 0.3492
Epoch 3/10
92/92 [==============================] - 20s 217ms/step - loss: 1.6758 - accuracy: 0.4221
Epoch 4/10
92/92 [==============================] - 20s 217ms/step - loss: 1.5413 - accuracy: 0.4786
Epoch 5/10
92/92 [==============================] - 20s 218ms/step - loss: 1.4513 - accuracy: 0.5099
Epoch 6/10
92/92 [========